<a href="https://colab.research.google.com/github/biswajitmohanty/genai/blob/main/LLM_with_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-groq gradio python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.5 MB/s eta 0:00:00


In [8]:
import os
import warnings
warnings.filterwarnings('ignore')
# Replace with your Groq API key
os.environ["GROQ_API_KEY"] = "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"

In [9]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
import gradio as gr
from IPython.display import display, Markdown

In [10]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0,
    max_tokens=200
)

## Define Summarization Prompt

In [11]:
summarize_prompt = ChatPromptTemplate.from_messages([
    ("system", "Summarize the input text in 2-3 sentences, capturing key points concisely."),
    ("human", "{input}")
])

## Summarization Function

In [12]:
def summarize(input):
    chain = summarize_prompt | llm
    response = chain.invoke({"input": input})
    return response.content

In [13]:
text = """The tower is 324 metres tall, about the same height as an 81-storey building, and the tallest structure in Paris..."""
print("Test Summary:", summarize(text))

Test Summary: The tower is 324 meters tall, equivalent to an 81-storey building, and holds the title of the tallest structure in Paris.


## Summarization Gradio Interface

In [14]:
gr.close_all()
demo1 = gr.Interface(
    fn=summarize,
    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
    outputs=[gr.Textbox(label="Result", lines=3)],
    title="Text Summarization with Groq",
    description="Summarize text using Groq's llama-3.1-8b-instant model."
)

In [15]:
demo1.launch(share=False)  # Run locally; use share=True for public URL (may require paid plan)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Define NER Prompt

In [16]:
ner_prompt = ChatPromptTemplate.from_messages([
    ("system", r"""Identify named entities (Person, Organization, Location) in the text. Return a JSON list of objects, each with 'word' (entity name), 'type' (Person, Organization, Location), 'start' (0-based index of first character), and 'end' (index after last character). Do not split names (e.g., 'arbaz' or 'mumbai' as one word). Examples: For 'My name is arbaz and i stay in mumbai', return: [[{{"word": "arbaz", "type": "Person", "start": 11, "end": 16}}, {{"word": "mumbai", "type": "Location", "start": 29, "end": 35}}]]."""),
    ("human", "{input}")
])

## NER Function

In [ ]:
def ner(input):
    chain = ner_prompt | llm
    response = chain.invoke({"input": input})
    print("Raw NER Response:", response.content)
    json_match = re.search(r'```json\n([\s\S]*?)\n```', response.content)
    try:
        entities = json.loads(json_match.group(1) if json_match else response.content)
        formatted_entities = [{"entity": e["type"], "word": e["word"], "start": e["start"], "end": e["end"]} for e in entities]
        for e in formatted_entities:
            substring = input[e["start"]:e["end"]].strip()
            if substring.lower() != e["word"].lower():
                print(f"Index warning: Expected {e['word']}, got {substring} at {e['start']}:{e['end']}")
                # Adjust indices via string matching
                match = re.search(r'\b' + re.escape(e["word"]) + r'\b', input, re.IGNORECASE)
                if match:
                    e["start"], e["end"] = match.start(), match.end()
                    print(f"Corrected indices for {e['word']}: {e['start']}:{e['end']}")
        return {"text": input, "entities": formatted_entities}
    except Exception as e:
        print("Parsing Error:", str(e))
        return {"text": input, "entities": []}

## Test NER

In [ ]:
test_text = "My name is arbaz and i stay in mumbai"
result = ner(test_text)
print("Test NER:", result)

In [ ]:
gr.close_all()
demo2 = gr.Interface(
    fn=ner,
    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
    outputs=[gr.HighlightedText(label="Text with entities")],
    title="NER with Groq",
    examples=["My name is arbaz and i stay in mumbai"]
)

In [ ]:
demo2.launch(share=False)  # Run locally